In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate 
from langchain_core.runnables import RunnableMap, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import CommaSeparatedListOutputParser
import pandas as pd 

1. Load the Dataset and inspect it

In [4]:
df = pd.read_csv('dataset/serie_a_forwards.csv')
df.head()

,full_name,country_code,role,season,team,mins_played,xG,goals,xA,assists,xT,xT_from_passes,xT_from_carries,won_dribblings,attempted_dribblings,won_dribblings_ratio,offensive_aero
0,Zlatan Ibrahimovic,SWE,FW,2021,Milan,1008,6.252449,8.0,2.012261,3.0,0.794123,0.635827,0.158296,5.0,9.0,0.5556,68.9437
1,Fabio Quagliarella,ITA,FW,2021,Sampdoria,1658,4.259178,4.0,1.837090,3.0,0.630270,0.585217,0.045052,11.0,23.0,0.4783,38.1917
2,Cristiano Ronaldo,PRT,FW,2021,Juventus,31,0.236411,NaN,0.028131,NaN,-0.018240,-0.017597,-0.000644,1.0,1.0,1.0000,NaN
3,Goran Pandev,MKD,FW,2021,Genoa,835,0.464921,NaN,1.186817,1.0,0.627832,0.349661,0.278171,10.0,22.0,0.4545,40.5377
4,Stefano Okaka,ITA,FW,2021,Udinese,20,0.033823,NaN,0.001525,1.0,0.016907,0.002524,0.014382,NaN,NaN,NaN,54.5864


In [5]:
df.columns

Index(['full_name', 'country_code', 'role', 'season', 'team', 'mins_played',
       'xG', 'goals', 'xA', 'assists', 'xT', 'xT_from_passes',
       'xT_from_carries', 'won_dribblings', 'attempted_dribblings',
       'won_dribblings_ratio', 'offensive_aero'],
      dtype='object')

2. Data preprocessing. Here you can add what you think might be useful to have a better seasonal analysis. Examples:
 - over / under performance indicators
 - normalizations
 - percentiles 
 - rankings


In [6]:
#calculate percentile of stats vs players in the same season
cols_to_pct = ['xG', 'goals', 'xA', 'assists', 'xT', 'xT_from_passes',
       'xT_from_carries', 'won_dribblings', 'attempted_dribblings',
       'won_dribblings_ratio', 'offensive_aero']

for stat in cols_to_pct:
    df[stat + '_percentile'] = df.groupby('season')[stat].rank(pct=True)


float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].round(2)

In [21]:
(   df
    .query('season == 2023')
    .sort_values('xG', ascending=False)
    [['full_name', 'season','xG', 'xG_percentile', 'goals', 'goals_percentile']]
    .head(10)
)

,full_name,season,xG,xG_percentile,goals,goals_percentile
496,Lautaro Martínez,2023,15.59,1.00,23.0,1.00
545,Dusan Vlahovic,2023,15.21,0.99,15.0,0.99
495,Marcus Thuram,2023,12.50,0.99,10.0,0.92
564,Olivier Giroud,2023,11.15,0.98,12.0,0.98
578,Romelu Lukaku,2023,10.32,0.98,10.0,0.92
575,Khvicha Kvaratskhelia,2023,10.26,0.97,10.0,0.92
489,Victor Osimhen,2023,10.07,0.96,11.0,0.96
532,Ciro Immobile,2023,9.62,0.96,6.0,0.78
570,Paulo Dybala,2023,9.40,0.95,12.0,0.98
574,Nikola Krstovic,2023,9.24,0.94,5.0,0.72



3. Write a prompt template to extract the informations you need, so player full name and season indication. 

Your goal here is to have the model generating a structured output, from which you can parse the relevant information (dict, list, comma separated string...)

Optional: To have a more flexible app that could also compare players, think of way to extract multiple players at once


In [43]:
PLAYER_INFO_TEMPLATE = '''
You are a words extractor from a given input. 
Your job is to extract words following instructions and providing them in a specific format.
Given a user question you must extract:
- Player full names
- Seasons

Provide the output as a list of dictionaries, where each pair player-season is a dict.
Follow this structure: 
[
        {{'full_name': '<player_name_1>' (str), 'season': <season_value_1> (int)}},
        ...{{'full_name': '<player_name_N>' (str), 'season': <season_value_N> (int)}}]

The season indication is a 4 digits number, indicating the year of the first matchday (e.g. 2020, 2021, 2022 and so on).
Note that the current season is 2023.

Example: Paulo Dybala seasonal performance in 2022. 
Output: [{{'full_name': 'Paulo Dybala', 'season': 2022}}]

Here is the input to process: 
{input}
'''


player_info_template = PromptTemplate(
        template=PLAYER_INFO_TEMPLATE, 
        input_variables=['input']
        )


llm = ChatOpenAI(model='gpt-3.5-turbo', timeout=4, max_retries=3, temperature=0)

info_chain = player_info_template | llm 


4. Test the extraction step! Look at the output and then find a way to parse it into a python data structure

In [44]:
input = 'Compare the seasons of Paulo Dybala in 2022 and Dusan Vlahovic in 2021'
result = info_chain.invoke({'input':input})
result.content
res = eval(result.content)

In [45]:
print(res, type(res), type(res[0]))

[{'full_name': 'Paulo Dybala', 'season': 2022}, {'full_name': 'Dusan Vlahovic', 'season': 2021}] <class 'list'> <class 'dict'>


5. Filter the player you want to analyze (select the correct season!) from the dataframe

In [46]:
players_to_filter = pd.DataFrame(res)
players_to_filter


,full_name,season
0,Paulo Dybala,2022
1,Dusan Vlahovic,2021


In [47]:
filtered_df = pd.merge(df, players_to_filter, on=['full_name', 'season'], how='inner')

6. Check your filtered df

In [48]:
filtered_df

,full_name,country_code,role,season,team,mins_played,xG,goals,xA,assists,xT,xT_from_passes,xT_from_carries,won_dribblings,attempted_dribblings,won_dribblings_ratio,offensive_aero
0,Dusan Vlahovic,SRB,FW,2021,Fiorentina,1861,12.051746,17.0,2.749618,2.0,0.660551,0.292195,0.368356,20.0,37.0,0.5405,68.7435
1,Dusan Vlahovic,SRB,FW,2021,Juventus,1077,5.668080,7.0,0.391012,1.0,0.172660,-0.073325,0.245985,11.0,27.0,0.4074,63.4608
2,Paulo Dybala,ARG,FW,2022,Roma,1751,9.692037,12.0,4.083198,6.0,2.271581,1.161230,1.110351,27.0,54.0,0.5000,28.7969


7. Transform your filtered dataframe into a list of dict (Hint: .to_dict(orient='records'))

In [49]:
players_data = filtered_df.to_dict(orient='records')

8. Write a prompt template to generate the final report given:
- user input
- extracted data
- metrics description (you can load them from  the script utils/descriptions.py)

Use the prompt techniques described into the lecture if you think they might fit. There is no right or wrong solution here.
Prompt engineering is a trial and error process, try different things and be creative!

In [85]:
from utils.descriptions import Descriptions
OUTPUT_TEMPLATE = '''
You are a soccer data analyst. You are provided with a user input and useful advanced metrics to answer the question.
If multiple players are provided, try to perform a comparison between them.

User input: {input}
Data: {data}

Use these definitions to better understand the metrics you are provided: 
{stat_description}

If a player has played in multiple teams in the same season, try compare the two situations when possible. 
Make your analysis as a report. 
Divide your analysis in two parts:

1. Creation: talk about assists and xA, xT and dribblings skill when notable
2. Finishing: talk about goals and xG and how they compare (over / under performance?) and about areial dominance (use AERO elo value!)
3. Overall conclusion, tell who is better overall in case of comparisons 

Make it a coherent text not a bullet point list. Put a title, and paragraph names well formatted.
'''


output_template = PromptTemplate(
        template=OUTPUT_TEMPLATE, 
        input_variables = ['input', 'data'], 
        partial_variables={'stat_description': Descriptions.stats}
        )

llm = ChatOpenAI(model='gpt-3.5-turbo', timeout=30, max_retries=3, temperature=0)

output_chain = output_template | llm 

9. Invoke the chain and see results!

In [86]:

report = output_chain.invoke(
    {
        'input': input, 
        'data': players_data,
        }
    )

In [87]:
from pprint import pprint
pprint(report.content)

('**Analysis of Paulo Dybala in 2022 vs Dusan Vlahovic in 2021**\n'
 '\n'
 '**Creation:**\n'
 'When comparing the creation aspect of the two players, Paulo Dybala in the '
 '2022 season stands out with his higher xA and assists numbers compared to '
 'Dusan Vlahovic in 2021. Dybala recorded an xA of 4.083 and 6 assists, '
 'showcasing his ability to create goal-scoring opportunities for his '
 'teammates. On the other hand, Vlahovic had an xA of 2.749 and 2 assists in '
 'his time at Fiorentina, and a lower xA of 0.391 and 1 assist during his '
 "stint at Juventus. Dybala's higher xA and assists numbers indicate his "
 'proficiency in setting up goals for his team.\n'
 '\n'
 'In terms of expected threat (xT), Dybala also outperformed Vlahovic in this '
 "aspect. Dybala's xT of 2.271 from passes and carries suggests that his "
 "actions significantly increased his team's scoring probability by moving the "
 'ball into more threatening zones. Vlahovic, on the other hand, had lower xT '
 

10. Make all togheter in a single chain

In [88]:
def parse_player_info(extracted_dict)->list:
    return eval(extracted_dict.content)

def filter_and_format_data(extracted_dict: dict)->dict:
    players_to_filter = pd.DataFrame(extracted_dict)
    filtered_df = pd.merge(df, players_to_filter, on=['full_name', 'season'], how='inner')
    return filtered_df.to_dict(orient='records')
    

str

In [89]:
seasonal_report_chain = (RunnableMap(
            steps={
                    "input": lambda x: x['input'],
                    "data": (
                        player_info_template 
                        | llm 
                        | parse_player_info 
                        | filter_and_format_data
                    ),
            }
))



{'steps': {'input': 'Paulo Dybala seasonal performance in 2021',
  'data': [{'full_name': 'Paulo Dybala',
    'country_code': 'ARG',
    'role': 'FW',
    'season': 2021,
    'team': 'Juventus',
    'mins_played': 2072,
    'xG': 8.1260417,
    'goals': 10.0,
    'xA': 4.4156602,
    'assists': 5.0,
    'xT': 1.9653445,
    'xT_from_passes': 0.740577,
    'xT_from_carries': 1.2247675,
    'won_dribblings': 47.0,
    'attempted_dribblings': 79.0,
    'won_dribblings_ratio': 0.5949,
    'offensive_aero': 34.7566}]}}

In [91]:
seasonal_report_chain = (RunnableMap(
            steps={
                    "input": lambda x: x['input'],
                    "data": (
                        player_info_template 
                        | llm 
                        | parse_player_info 
                        | filter_and_format_data
                    ),
                })
            | RunnableLambda(lambda x: x['steps'])
            | output_template
            | llm 
            )


In [93]:
res = seasonal_report_chain.invoke({'input': "Compare the seasonal performance of Cristiano Ronaldo and Romelu Lukaku in 2020"})
pprint(res)

AIMessage(content="**Analysis of Seasonal Performance: Cristiano Ronaldo vs Romelu Lukaku in 2020**\n\n**Creation:**\nWhen it comes to the creation aspect of their game, Romelu Lukaku had a standout season in 2020 with 11 assists compared to Cristiano Ronaldo's 3 assists. Lukaku's xA (Expected Assists) value of 6.34 also indicates his ability to create goal-scoring opportunities for his teammates. On the other hand, Ronaldo's xA of 3.42 shows his contribution in this aspect but falls short compared to Lukaku. \n\nIn terms of expected threat (xT), both players had similar xT values, with Lukaku slightly edging out Ronaldo. Lukaku's xT from passes and carries were 0.92 and 2.10 respectively, while Ronaldo's were 0.92 and 2.88. This suggests that both players were equally effective in increasing their team's scoring probability through their actions on the field.\n\nDribbling is another key aspect of creation, and Ronaldo had a higher success rate in dribbling with a 61.05% success rate c